## imports and files

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
__file__ = globals()['_dh'][0]
project_dir = __file__ = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(
    project_dir, '..', '..', '..')))

from experiments.utils.constants import (
    PIPLINES_PATH,
    PIPELINE_PROFILING_CONFIGS_PATH,
    PIPELINE_PROFILING_RESULTS_PATH
)
from experiments.utils.parser import Parser

In [2]:
from json import load

series = 1
config_key_mapper = "key_config_mapper.csv"

series_path = os.path.join(
    PIPELINE_PROFILING_RESULTS_PATH,
    'series',
    str(series))

loader = Loader(
    series_path=series_path, config_key_mapper=config_key_mapper, model_name='video', type_of='pipeline')

In [3]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config) 

File name: 0.yaml
{   'benchmark_duration': 1,
    'metadata': 'Trying out the pipeline',
    'mode': 'exponential',
    'nodes': [   {   'cpu_request': ['8'],
                     'data_type': 'image',
                     'max_batch_size': ['1'],
                     'max_batch_time': ['1'],
                     'memory_request': ['10Gi'],
                     'model_variants': ['yolov5n'],
                     'node_name': 'yolo',
                     'num_interop_threads': ['1'],
                     'num_threads': ['1'],
                     'replicas': [1],
                     'use_threading': 'True'},
                 {   'cpu_request': ['8'],
                     'data_type': 'text',
                     'max_batch_size': ['1'],
                     'max_batch_time': ['1'],
                     'memory_request': ['10Gi'],
                     'model_variants': ['resnet18'],
                     'node_name': 'resnet-human',
                     'num_interop_threads': ['1'],
   

## config map reader

In [4]:
key_config_df = loader.key_config_mapper()
display(key_config_df)
key_config_df.keys()

,experiment_id,pipeline_name,load,load_duration,series,metadata,mode,data_type,benchmark_duration,task_0_node_name,...,task_0_max_batch_size,task_0_max_batch_time,task_0_replica,task_1_node_name,task_1_model_variant,task_1_cpu_request,task_1_memory_request,task_1_max_batch_size,task_1_max_batch_time,task_1_replica
0,1,video,1,30,1,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,8,10Gi,1,1,1


Index(['experiment_id', 'pipeline_name', 'load', 'load_duration', 'series',
       'metadata', 'mode', 'data_type', 'benchmark_duration',
       'task_0_node_name', 'task_0_model_variant', 'task_0_cpu_request',
       'task_0_memory_request', 'task_0_max_batch_size',
       'task_0_max_batch_time', 'task_0_replica', 'task_1_node_name',
       'task_1_model_variant', 'task_1_cpu_request', 'task_1_memory_request',
       'task_1_max_batch_size', 'task_1_max_batch_time', 'task_1_replica'],
      dtype='object')

In [5]:
experiment_results = loader.get_experiment_detail(1)
display(experiment_results)
experiment_results.keys()


,experiment_id,pipeline_name,load,load_duration,series,metadata,mode,data_type,benchmark_duration,task_0_node_name,...,task_0_max_batch_size,task_0_max_batch_time,task_0_replica,task_1_node_name,task_1_model_variant,task_1_cpu_request,task_1_memory_request,task_1_max_batch_size,task_1_max_batch_time,task_1_replica
0,1,video,1,30,1,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,8,10Gi,1,1,1


Index(['experiment_id', 'pipeline_name', 'load', 'load_duration', 'series',
       'metadata', 'mode', 'data_type', 'benchmark_duration',
       'task_0_node_name', 'task_0_model_variant', 'task_0_cpu_request',
       'task_0_memory_request', 'task_0_max_batch_size',
       'task_0_max_batch_time', 'task_0_replica', 'task_1_node_name',
       'task_1_model_variant', 'task_1_cpu_request', 'task_1_memory_request',
       'task_1_max_batch_size', 'task_1_max_batch_time', 'task_1_replica'],
      dtype='object')

In [6]:
filenames = loader.get_result_file_names()
filenames

['1.json', '1.txt']

## Read resutls

In [7]:
results = loader.read_results()
print(results.keys())

dict_keys(['1'])


In [8]:
flattened_results = loader.flatten_results(results['1']['responses'])
flattened_results

[{'times': {'request': {'sending': 1678493827.7029424,
    'arrival': 1678493827.8235018},
   'models': {'resnet-human': {'arrival': 1678493827.7719069,
     'serving': 1678493827.8016603},
    'yolo': {'arrival': 1678493827.7138805, 'serving': 1678493827.7611244}}},
  'model_name': 'resnet-human',
  'outputs': []},
 {'times': {'request': {'sending': 1678493828.7176595,
    'arrival': 1678493828.8066902},
   'models': {'resnet-human': {'arrival': 1678493828.7767565,
     'serving': 1678493828.802546},
    'yolo': {'arrival': 1678493828.7239683, 'serving': 1678493828.7703483}}},
  'model_name': 'resnet-human',
  'outputs': []},
 {'times': {'request': {'sending': 1678493828.384333,
    'arrival': 1678493828.4760258},
   'models': {'resnet-human': {'arrival': 1678493828.4451869,
     'serving': 1678493828.4718623},
    'yolo': {'arrival': 1678493828.3914351, 'serving': 1678493828.438075}}},
  'model_name': 'resnet-human',
  'outputs': []},
 {'times': {'request': {'sending': 1678493831.177

In [9]:
experiment_id = 1
flattened_results = loader.flatten_results(
    results[str(experiment_id)]['responses'])
latencies, timeout_count = loader.latency_calculator(
    flattened_results)
print(f'{timeout_count=}')
print(pd.DataFrame(latencies))

timeout_count=0
    client_to_pipeline_latencies  pipeline_to_client_latencies  \
0                       0.010938                      0.021842   
1                       0.006309                      0.004144   
2                       0.007102                      0.004164   
3                       0.007621                      0.005272   
4                       0.006409                      0.004095   
5                       0.007948                      0.004235   
6                       0.008269                      0.004039   
7                       0.007210                      0.004080   
8                       0.007606                      0.004330   
9                       0.006620                      0.003834   
10                      0.008009                      0.003960   
11                      0.007557                      0.004118   
12                      0.006952                      0.004115   
13                      0.007147                      0.0074

In [10]:
summary = loader.latency_summary(latencies=latencies)
pd.DataFrame(list(summary.items()))

,0,1
0,client_to_pipeline_latencies_avg,7.326078e-03
1,client_to_pipeline_latencies_p99,1.027413e-02
2,client_to_pipeline_latencies_p50,7.168412e-03
3,client_to_pipeline_latencies_var,9.092366e-07
4,client_to_pipeline_latencies_max,1.093817e-02
5,client_to_pipeline_latencies_min,6.161928e-03
6,pipeline_to_client_latencies_avg,4.925585e-03
7,pipeline_to_client_latencies_p99,1.765937e-02
8,pipeline_to_client_latencies_p50,4.104972e-03
9,pipeline_to_client_latencies_var,1.044231e-05


In [13]:
results = loader.result_processing()
# print(results)
# print(list(results.columns))
for item in list(results.columns):
    if 'task_0' in item:
        print(item)

task_0_model_latencies_avg
task_0_model_latencies_p99
task_0_model_latencies_p50
task_0_model_latencies_var
task_0_model_latencies_max
task_0_model_latencies_min
task_0_to_task_1_latencies_avg
task_0_to_task_1_latencies_p99
task_0_to_task_1_latencies_p50
task_0_to_task_1_latencies_var
task_0_to_task_1_latencies_max
task_0_to_task_1_latencies_min
task_0_cpu_usage_count_avg
task_0_cpu_usage_count_p99
task_0_cpu_usage_count_p50
task_0_cpu_usage_count_var
task_0_cpu_usage_count_max
task_0_cpu_usage_count_min
task_0_cpu_usage_rate_avg
task_0_cpu_usage_rate_p99
task_0_cpu_usage_rate_p50
task_0_cpu_usage_rate_var
task_0_cpu_usage_rate_max
task_0_cpu_usage_rate_min
task_0_cpu_throttled_count_avg
task_0_cpu_throttled_count_p99
task_0_cpu_throttled_count_p50
task_0_cpu_throttled_count_var
task_0_cpu_throttled_count_max
task_0_cpu_throttled_count_min
task_0_cpu_throttled_rate_avg
task_0_cpu_throttled_rate_p99
task_0_cpu_throttled_rate_p50
task_0_cpu_throttled_rate_var
task_0_cpu_throttled_rate_ma

In [15]:
key_config_df = loader.key_config_mapper()
print(key_config_df.columns)
# selected_key_config_df =  key_config_df[key_config_df['model_variant_1'] == 'yolov5n']
# selected_key_config_df
key_config_df

Index(['experiment_id', 'pipeline_name', 'load', 'load_duration', 'series',
       'metadata', 'mode', 'data_type', 'benchmark_duration',
       'task_0_node_name', 'task_0_model_variant', 'task_0_cpu_request',
       'task_0_memory_request', 'task_0_max_batch_size',
       'task_0_max_batch_time', 'task_0_replica', 'task_1_node_name',
       'task_1_model_variant', 'task_1_cpu_request', 'task_1_memory_request',
       'task_1_max_batch_size', 'task_1_max_batch_time', 'task_1_replica'],
      dtype='object')


,experiment_id,pipeline_name,load,load_duration,series,metadata,mode,data_type,benchmark_duration,task_0_node_name,...,task_0_max_batch_size,task_0_max_batch_time,task_0_replica,task_1_node_name,task_1_model_variant,task_1_cpu_request,task_1_memory_request,task_1_max_batch_size,task_1_max_batch_time,task_1_replica
0,1,video,1,30,1,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,8,10Gi,1,1,1


In [18]:
experiment_ids = key_config_df[
    key_config_df['task_0_node_name'] == 'yolo']['experiment_id'].tolist()
# model = 'resnet-human'
# pod = 'pod1'
task = 'task_0'
metadata_columns = ['task_0_node_name', 'task_0_cpu_request']
results_columns = [
    f'{task}_model_latencies_p99',
    f'{task}_throughput_max']
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns)
display(output)


,task_0_node_name,task_0_cpu_request,task_0_model_latencies_p99,task_0_throughput_max
0,yolo,8,0.070353,1.034483


In [16]:
# TODO add figures